In [1]:
import pandas as pd
#Load file
df = pd.read_excel('/content/Base_FINAL.xlsx')

In [2]:
# Filters
df = df[(df['AÑO'] >= 2018) & (~df['TIPO'].isin(['EST', 'OTRAS']))]

In [3]:
import time
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.utils import resample
from tqdm import tqdm

# Prepare features and labels
X_features = df.drop(columns=['TIPO', 'AÑO', 'MORENA', 'PRIANPRD', 'OTROS'])
y = df[['MORENA', 'PRIANPRD', 'OTROS']]

# Convert categorical 'EDO' column to numeric using one-hot encoding
X_features = pd.get_dummies(X_features, columns=['EDO'], drop_first=True)

# Define function to calculate MAPE
def mean_absolute_percentage_error(y_true, y_pred, threshold=1e-2):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    mask = y_true > threshold
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

# Initialize dictionaries to store metrics and predictions
mse_scores = {}
mae_scores = {}
mape_scores = {}
trained_models = {}

# Multi-output models
multi_output_models = {
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'XGBoost': xgb.XGBRegressor(n_estimators=100, random_state=42),
    'LightGBM': lgb.LGBMRegressor(n_estimators=100, random_state=42)
}

# Single-output models
single_output_models = {
    'Linear Regression': LinearRegression(),
    'SVR': SVR(kernel='rbf'),
    'KNN': KNeighborsRegressor(n_neighbors=3)
}

# Train and evaluate multi-output models using MultiOutputRegressor on the entire dataset
for name, model in multi_output_models.items():
    multi_model = MultiOutputRegressor(model)
    multi_model.fit(X_features, y)
    preds = multi_model.predict(X_features)
    mse_scores[name] = mean_squared_error(y, preds)
    mae_scores[name] = mean_absolute_error(y, preds)
    mape_scores[name] = mean_absolute_percentage_error(y, preds)
    trained_models[name] = multi_model

# Train and evaluate single-output models on the entire dataset
for name, model in single_output_models.items():
    model_mse = []
    model_mae = []
    model_mape = []
    preds_list = []
    trained_model_list = []

    for i in range(y.shape[1]):
        model.fit(X_features, y.iloc[:, i])
        preds = model.predict(X_features)
        preds_list.append(preds)
        model_mse.append(mean_squared_error(y.iloc[:, i], preds))
        model_mae.append(mean_absolute_error(y.iloc[:, i], preds))
        model_mape.append(mean_absolute_percentage_error(y.iloc[:, i], preds))
        trained_model_list.append(model)

    mse_scores[name] = np.mean(model_mse)
    mae_scores[name] = np.mean(model_mae)
    mape_scores[name] = np.mean(model_mape)
    trained_models[name] = trained_model_list

# Select the top three models based on the lowest errors
sorted_models = sorted(mae_scores.items(), key=lambda item: (item[1], mape_scores[item[0]], mse_scores[item[0]]))
top_three_models = [model[0] for model in sorted_models[:3]]

# Create a DataFrame to display the errors of the top three models
top_three_errors = pd.DataFrame(
    [(model, mse_scores[model], mae_scores[model], mape_scores[model]) for model in top_three_models],
    columns=['Model', 'MSE', 'MAE', 'MAPE']
)

# Display the DataFrame with the top three models and their errors
print("Top Three Models with Lowest Errors:")
print(top_three_errors)

# Timing the process
start_time = time.time()

# Initialize dictionary to store predictions per EDO
predictions_per_edo = {}
ci_per_edo = {}

# Get unique labels in the 'EDO' column
unique_edos = df['EDO'].unique()

# Number of bootstrap samples
n_bootstrap_samples = 100  # Reduced to 100 for faster computation

# General prediction
general_predictions = {}
general_ci = {}

# Calculate general predictions and confidence intervals
for name in top_three_models:
    multi_model = trained_models[name]
    preds_samples = []

    for _ in tqdm(range(n_bootstrap_samples), desc=f"Bootstrap sampling for general {name}"):
        X_resampled = resample(X_features)
        preds_resampled = multi_model.predict(X_resampled)
        preds_samples.append(preds_resampled.mean(axis=0))

    preds_samples = np.array(preds_samples)
    preds_mean = preds_samples.mean(axis=0)
    preds_lower = np.percentile(preds_samples, 2.5, axis=0)
    preds_upper = np.percentile(preds_samples, 97.5, axis=0)

    general_predictions[name] = preds_mean
    general_ci[name] = (preds_lower, preds_upper)

# Iterate over each unique 'EDO' label
for edo in tqdm(unique_edos, desc="Processing EDOs"):
    # Create a dummy data point for prediction for the given EDO
    X_new_period_edo = X_features.mean().to_frame().T
    X_new_period_edo['EDO_' + str(edo)] = 1  # Add one-hot encoded EDO column

    # Ensure the new period data has the same columns as the training set
    X_new_period_edo = X_new_period_edo.reindex(columns=X_features.columns, fill_value=0)

    # Store predictions and confidence intervals for the top three models
    for name in top_three_models:
        multi_model = trained_models[name]

        # Bootstrap sampling to calculate confidence intervals
        preds_samples = []
        for _ in range(n_bootstrap_samples):
            preds_resampled = multi_model.predict(X_new_period_edo)
            preds_samples.append(preds_resampled.flatten())  # Removed normalization step here

        preds_samples = np.array(preds_samples)
        preds_mean = preds_samples.mean(axis=0)
        preds_lower = np.percentile(preds_samples, 2.5, axis=0)
        preds_upper = np.percentile(preds_samples, 97.5, axis=0)

        predictions_per_edo.setdefault(name, []).append(preds_mean)
        ci_per_edo.setdefault(name, []).append((preds_lower, preds_upper))

# Timing end
end_time = time.time()
total_time = end_time - start_time

# Convert general predictions to DataFrames and add labels
general_predictions_df = {name: pd.DataFrame([preds], columns=['MORENA', 'PRIANPRD', 'OTROS']) for name, preds in general_predictions.items()}

# Convert EDO predictions to DataFrames and add labels
predictions_df_per_edo = {name: pd.DataFrame(preds, index=unique_edos, columns=['MORENA', 'PRIANPRD', 'OTROS']) for name, preds in predictions_per_edo.items()}

# Display the general predictions and confidence intervals
for name, df in general_predictions_df.items():
    print(f"\n{name} general predictions for the next period:")
    print(df)
    lower_ci = general_ci[name][0]
    upper_ci = general_ci[name][1]
    print(f"95% Confidence Intervals for {name}:")
    print(f"Lower - MORENA: {lower_ci[0]:.4f}, PRIANPRD: {lower_ci[1]:.4f}, OTROS: {lower_ci[2]:.4f}")
    print(f"Upper - MORENA: {upper_ci[0]:.4f}, PRIANPRD: {upper_ci[1]:.4f}, OTROS: {upper_ci[2]:.4f}")

# Display the DataFrames with EDO predictions and confidence intervals
for name, df in predictions_df_per_edo.items():
    print(f"\n{name} predictions per EDO for the next period:")
    print(df)
    print(f"95% Confidence Intervals for {name}:")
    for i, edo in enumerate(unique_edos):
        lower_ci = ci_per_edo[name][i][0]
        upper_ci = ci_per_edo[name][i][1]
        print(f"{edo}: Lower - MORENA: {lower_ci[0]:.4f}, PRIANPRD: {lower_ci[1]:.4f}, OTROS: {lower_ci[2]:.4f}")
        print(f"       Upper - MORENA: {upper_ci[0]:.4f}, PRIANPRD: {upper_ci[1]:.4f}, OTROS: {upper_ci[2]:.4f}")

print(f"Total execution time: {total_time:.2f} seconds")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000079 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 100
[LightGBM] [Info] Number of data points in the train set: 220, number of used features: 4
[LightGBM] [Info] Start training from score 0.527188
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

Processing EDOs: 100%|██████████| 33/33 [01:58<00:00,  3.58s/it]


XGBoost general predictions for the next period:
     MORENA  PRIANPRD     OTROS
0  0.526353   0.40496  0.067281
95% Confidence Intervals for XGBoost:
Lower - MORENA: 0.5131, PRIANPRD: 0.3878, OTROS: 0.0612
Upper - MORENA: 0.5401, PRIANPRD: 0.4188, OTROS: 0.0731

Random Forest general predictions for the next period:
     MORENA  PRIANPRD     OTROS
0  0.525293  0.407392  0.065058
95% Confidence Intervals for Random Forest:
Lower - MORENA: 0.5107, PRIANPRD: 0.3945, OTROS: 0.0595
Upper - MORENA: 0.5386, PRIANPRD: 0.4191, OTROS: 0.0702

Gradient Boosting general predictions for the next period:
    MORENA  PRIANPRD     OTROS
0  0.52713  0.404455  0.066871
95% Confidence Intervals for Gradient Boosting:
Lower - MORENA: 0.5191, PRIANPRD: 0.3946, OTROS: 0.0607
Upper - MORENA: 0.5373, PRIANPRD: 0.4164, OTROS: 0.0716

XGBoost predictions per EDO for the next period:
      MORENA  PRIANPRD     OTROS
1   0.592179  0.323669  0.081491
2   0.588066  0.305025  0.081491
3   0.586189  0.323622  0.081